In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
import openai
import os
import json

In [3]:
def fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location):

    # Fill the 'Name' field
    name_field = driver.find_element(By.ID, "fin_name")
    name_field.clear()  # Clear any pre-filled text
    name_field.send_keys(name)

    # Fill the 'Gender' field 
    if gender.lower() == "male":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="male"]')
    elif gender.lower() == "female":
        gender_radio = driver.find_element(By.XPATH, '//input[@name="gender" and @value="female"]')
    gender_radio.click()

    # Fill the 'Day' field
    day_select = Select(driver.find_element(By.ID, "fin_day"))
    day_select.select_by_value(str(day))

    # Fill the 'Month' field
    month_select = Select(driver.find_element(By.ID, "fin_month"))
    month_select.select_by_value(str(month))

    # Fill the 'Year' field
    year_select = Select(driver.find_element(By.ID, "fin_year"))
    year_select.select_by_value(str(year))

    # Fill the 'Hour' field
    hour_select = Select(driver.find_element(By.ID, "fin_hour"))
    hour_select.select_by_value(str(hour))

    # Fill the 'Minute' field
    minute_select = Select(driver.find_element(By.ID, "fin_min"))
    minute_select.select_by_value(str(minute))

    # Fill the 'AM/PM' field
    ampm_select = Select(driver.find_element(By.ID, "fin_apm"))
    ampm_select.select_by_value(ampm)

    # Fill the 'Location' field
    location_input = driver.find_element(By.ID, "fin_location")
    
    location_input.clear()
    location_input.send_keys(location)
    time.sleep(2)  

    location_input.send_keys(Keys.DOWN) 
    time.sleep(1)
    location_input.send_keys(Keys.RETURN)  

    # Submit the form
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and contains(@class, 'btn-horoscope-form-submit')]")
    submit_button.click()

    # Wait for the page to load after submission 
    time.sleep(5)

In [4]:
def scrape_planet_positions(driver):

    table_rows = driver.find_elements(By.XPATH, "//table[@class='table table-bordered t-sm no-margin']/tbody/tr")
    
    planet_positions = []
    
    for row in table_rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        
        if len(columns) == 6:
            planet = row.find_element(By.XPATH, ".//th").text.strip().split()[0]  
            position = columns[0].text.strip()
            degrees = columns[1].text.strip()
            rasi = columns[2].text.strip().split()[0]  
            rasi_lord = columns[3].text.strip()
            nakshatra = columns[4].text.strip()
            nakshatra_lord = columns[5].text.strip()
            
            planet_positions.append({
                "Planet": planet,
                "Position": position,
                "Degrees": degrees,
                "Rasi": rasi,
                "Rasi Lord": rasi_lord,
                "Nakshatra": nakshatra,
                "Nakshatra Lord": nakshatra_lord
            })
    
    planet_positions_df = pd.DataFrame(planet_positions)
    return planet_positions_df

In [ ]:
if __name__ == "__main__":
    chrome_options = Options()
    chrome_options.add_argument("--headless")         # This makes the browser run in the background
    # chrome_options.add_argument("--disable-gpu")    # Disable GPU acceleration (optional)
    # chrome_options.add_argument("--no-sandbox")     # Fixes issues on some systems (optional)

    driver = webdriver.Chrome(options=chrome_options)

    # Open the webpage
    driver.get("https://www.prokerala.com/astrology/birth-chart/")

    # Wait for the page to load
    time.sleep(2)

    # Enter details 
    name = input("Enter your name: ")
    gender = input("Enter your gender (male/female): ")
    day = input("Enter your birth day (1-31): ")
    month = input("Enter your birth month (1-12): ")
    year = input("Enter your birth year (e.g., 1990): ")
    hour = input("Enter your birth hour (1-12): ")
    minute = input("Enter your birth minute (0-59): ")
    ampm = input("Enter AM/PM (am/pm): ")
    location = input("Enter your birth place (city): ")

    fill_form(driver, name, gender, day, month, year, hour, minute, ampm, location)


    planet_positions_df = scrape_planet_positions(driver)
    print(planet_positions_df)
    planet_positions_df.to_csv("planet_positions.csv", index=False)

    driver.quit()

      Planet  Position  Degrees       Rasi Rasi Lord          Nakshatra  \
0        Sun   142° 4'   22° 4′      Simha       Sun     Purva Phalguni   
1       Moon   25° 25'  25° 25′      Mesha      Mars            Bharani   
2    Mercury  166° 38'  16° 38′      Kanya   Mercury              Hasta   
3      Venus  110° 56'  20° 56′      Karka      Moon           Ashlesha   
4       Mars   246° 1'    6° 1′      Dhanu   Jupiter              Moola   
5    Jupiter   77° 16'  17° 16′    Mithuna   Mercury              Ardra   
6     Saturn   50° 46'  20° 46′  Vrishabha     Venus             Rohini   
7  Ascendant  337° 39'   7° 39′      Meena   Jupiter  Uttara Bhadrapada   
8       Rahu   68° 32'   8° 32′    Mithuna   Mercury              Ardra   
9       Ketu  248° 32'   8° 32′      Dhanu   Jupiter              Moola   

  Nakshatra Lord  
0          Venus  
1          Venus  
2           Moon  
3        Mercury  
4           Ketu  
5           Rahu  
6           Moon  
7         Saturn  
8  

In [16]:
# === Generic Model (Commented Out) ===
# def format_kundli_naturally(df):
#     descriptions = []
#     for _, row in df.iterrows():
#         line = (
#             f"The planet {row['Planet']} is positioned at {row['Position']} in the sign of {row['Rasi']} "
#             f"(ruled by {row['Rasi Lord']}). It is in the nakshatra {row['Nakshatra']}, governed by {row['Nakshatra Lord']}."
#         )
#         descriptions.append(line)
#     return "\n".join(descriptions)
# 
# # === STEP 3: Dynamic prompt creation ===
# def create_thoughtful_astrology_prompt(kundli_description, user_input):
#     return f"""
# You are a wise and introspective Vedic astrologer known for your emotional intelligence and deep spiritual insights.
# 
# Below is the person's planetary data:
# 
# {kundli_description}
# 
# They have shared the following question or concern:
# "{user_input}"
# 
# Please follow this thoughtful approach:
# 
# 1. **Identify the planets most relevant** to their concern or life area.
# 2. **Think through their placements step by step**, including sign, nakshatra, and ruling planets. Reflect on how these elements might interact.
# 3. **Reason through potential life patterns or emotional dynamics** that could emerge from these placements.
# 4. **Gently synthesize** this into a warm, spiritually grounded interpretation. Connect symbolism to emotional experience.
# 5. End with a **short open-ended question** that encourages self-reflection and personal growth.
# 
# Avoid vague advice or prediction. Focus on clarity, compassion, and empowering reflection.
# """
# 
# 
# # === INTERACTIVE CHAT LOOP ===
# def run_kundli_conversation(df):
#     client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# 
#     kundli_description = format_kundli_naturally(df)
#     print("\n🌟 Kundli Summary:")
#     print(kundli_description)
# 
#     print("\n✨ Ask anything about your life — career, emotions, family, patterns, etc.")
#     print("Type 'exit' to end the session.\n")
# 
#     while True:
#         user_input = input("🧘 What would you like to explore?: ").strip()
#         if user_input.lower() in {"exit", "quit"}:
#             print("🙏 Ending session. Take care!")
#             break
# 
#         gpt_prompt = create_thoughtful_astrology_prompt(kundli_description, user_input)
# 
#         try:
#             response = client.chat.completions.create(
#                 model="gpt-4o",  # or use "gpt-3.5-turbo" if needed
#                 messages=[{"role": "user", "content": gpt_prompt}]
#             )
#             print("\n🔮 Insight:\n")
#             print(response.choices[0].message.content)
#             print("\n--- Ask another question or type 'exit' ---\n")
# 
#         except Exception as e:
#             print("⚠️ Something went wrong:", e)
# 
# # === LAUNCH THE INTERACTIVE SESSION ===
# run_kundli_conversation(planet_positions_df)


🌟 Kundli Summary:
The planet Sun is positioned at 138° 19' in the sign of Simha (ruled by Sun). It is in the nakshatra Purva Phalguni, governed by Venus.
The planet Moon is positioned at 265° 22' in the sign of Dhanu (ruled by Jupiter). It is in the nakshatra Purva Ashadha, governed by Venus.
The planet Mercury is positioned at 164° 54' in the sign of Kanya (ruled by Mercury). It is in the nakshatra Hasta, governed by Moon.
The planet Venus is positioned at 142° 28' in the sign of Simha (ruled by Sun). It is in the nakshatra Purva Phalguni, governed by Venus.
The planet Mars is positioned at 184° 47' in the sign of Tula (ruled by Venus). It is in the nakshatra Chitra, governed by Mars.
The planet Jupiter is positioned at 223° 23' in the sign of Vrischika (ruled by Mars). It is in the nakshatra Anuradha, governed by Saturn.
The planet Saturn is positioned at 328° 15' in the sign of Kumbha (ruled by Saturn). It is in the nakshatra Purva Bhadrapada, governed by Jupiter.
The planet Ascend

In [ ]:
# === Format planetary data ===

def format_kundli_naturally(df):
    descriptions = []
    for _, row in df.iterrows():
        line = (
            f"The planet {row['Planet']} is positioned at {row['Position']} in the sign of {row['Rasi']} "
            f"(ruled by {row['Rasi Lord']}). It is in the nakshatra {row['Nakshatra']}, governed by {row['Nakshatra Lord']}."
        )
        descriptions.append(line)
    return "\n".join(descriptions)

# === Create prompt based on vibe ===
def create_prompt_by_vibe(kundli_description, user_input, vibe):
    base = f"\nBelow is the person's planetary data:\n\n{kundli_description}\n\nThey shared this question or concern:\n\"{user_input}\"\n"

    if vibe == "straightforward":
        return f"""
You are a clear, concise Vedic astrologer.

{base}

Please give a direct and accurate reading based on the relevant planetary placements. Keep it to the point and practical, avoiding vague language or fluff.
"""

    elif vibe == "reflective":
        return f"""
You are a wise and introspective Vedic astrologer known for emotional intelligence and gentle insight.

{base}

Step through your thought process. Identify the most important placements. Then share a warm, thoughtful interpretation.

End with a short, open-ended question for personal reflection.
"""

    elif vibe == "deep spiritual":
        return f"""
You are a poetic, mystical Vedic astrologer — a spiritual guide who speaks in symbolism, metaphor, and ancient wisdom.

{base}

Interpret the chart as a map of the soul. Weave together planets, signs, and nakshatras into a story of destiny, karma, and inner truth.

End with a reflective question that invites the user to meditate on this truth.
"""

    elif vibe == "therapist":
        return f"""
You are a modern, compassionate therapist who blends Vedic astrology with emotional psychology.

{base}

First, explore how planetary placements may influence emotional patterns, inner conflicts, or growth edges. Speak gently and empathetically. Validate their experience.

End with a therapeutic question that encourages healing or self-compassion.
"""

    elif vibe == "best_friend":
        return f"""
You are the user's best friend — the one who knows their astrology chart *and* their favorite snacks.

{base}

Speak casually, lovingly, and with emotional honesty. Use informal tone, little emojis or warmth if needed. Help them make sense of the situation, validate their feelings, and gently point out truths you know they need to hear.

Feel free to use phrases like "babe," "real talk," or "you already know this" — but keep it heartfelt.

End with a cheeky or supportive question that encourages a moment of real self-reflection or love.
"""

    else:
        # fallback: reflective
        return create_prompt_by_vibe(kundli_description, user_input, "reflective")

# === Interactive chat loop ===
def run_kundli_conversation(df):
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    

    kundli_description = format_kundli_naturally(df)
    print("\n🌟 Kundli Summary:")
    print(kundli_description)

    print("\n🌈 Choose your vibe:")
    print("[1] Straightforward")
    print("[2] Reflective")
    print("[3] Deep Spiritual")
    print("[4] Modern Therapist-style")
    print("[5] Talking to Your Best Friend 💖")

    vibe_choice = input(
    "\n🎨 Choose your vibe:\n"
    "[1] Straightforward — clear and practical\n"
    "[2] Reflective — gentle and introspective\n"
    "[3] Deep Spiritual — poetic, mystical guidance\n"
    "[4] Modern Therapist — emotionally grounded and supportive\n"
    "[5] Best Friend — casual, warm, and validating 💖\n"
    "👉 Enter 1–5 to set the tone of your session: "
    ).strip()

    vibe_map = {
        "1": "straightforward",
        "2": "reflective",
        "3": "deep spiritual",
        "4": "therapist",
        "5": "best_friend"
    }

    vibe = vibe_map.get(vibe_choice, "reflective")
    print(f"\n✨ You selected: {vibe.replace('_', ' ').title()} Mode\n")

    print("Type 'exit' to end the session.\n")

    while True:
        user_input = input("🧘 What would you like to explore?: ").strip()
        if user_input.lower() in {"exit", "quit"}:
            print("🙏 Ending session. Take care!")
            break

        gpt_prompt = create_prompt_by_vibe(kundli_description, user_input, vibe)

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": gpt_prompt}]
            )
            print("\n🔮 Here is your personalised Vedic Astrology 🔮: \n")
            print(response.choices[0].message.content)
            collect_user_feedback(response.choices[0].message.content, vibe)
            print("\n--- Ask another question or type 'exit' ---\n")

        except Exception as e:
            print("Something went wrong:", e)
    
# === Ask for feedback ===
def collect_user_feedback(vibe_used):

    tone_rating = input(f"Rate the tone matching the '{vibe_used}' vibe (1-5): ")
    helpfulness = input("Rate how helpful the response felt (1-5): ")
    personalization = input("Rate how personalized it felt (1-5): ")
    comments = input("Any thoughts or suggestions? ")

    feedback = {
        "vibe": vibe_used,
        "tone_rating": tone_rating,
        "helpfulness": helpfulness,
        "personalization": personalization,
        "comments": comments,
    }

    with open("user_feedback_log.json", "a") as f:
        f.write(json.dumps(feedback) + "\n")

    print("✅ Thank you for your feedback!")


In [31]:
run_kundli_conversation(planet_positions_df)


🌟 Kundli Summary:
The planet Sun is positioned at 142° 4' in the sign of Simha (ruled by Sun). It is in the nakshatra Purva Phalguni, governed by Venus.
The planet Moon is positioned at 25° 25' in the sign of Mesha (ruled by Mars). It is in the nakshatra Bharani, governed by Venus.
The planet Mercury is positioned at 166° 38' in the sign of Kanya (ruled by Mercury). It is in the nakshatra Hasta, governed by Moon.
The planet Venus is positioned at 110° 56' in the sign of Karka (ruled by Moon). It is in the nakshatra Ashlesha, governed by Mercury.
The planet Mars is positioned at 246° 1' in the sign of Dhanu (ruled by Jupiter). It is in the nakshatra Moola, governed by Ketu.
The planet Jupiter is positioned at 77° 16' in the sign of Mithuna (ruled by Mercury). It is in the nakshatra Ardra, governed by Rahu.
The planet Saturn is positioned at 50° 46' in the sign of Vrishabha (ruled by Venus). It is in the nakshatra Rohini, governed by Moon.
The planet Ascendant is positioned at 337° 39' 

In [ ]:
# === Evaluate Feedback Accuracy ===
def compute_feedback_accuracy(feedback_file="user_feedback_log.json"):
    import json

    try:
        with open(feedback_file, "r") as f:
            lines = f.readlines()
            feedbacks = [json.loads(line) for line in lines]
    except FileNotFoundError:
        print("Feedback file not found.")
        return

    total = len(feedbacks)
    if total == 0:
        print("No feedback data found.")
        return

    # Define correctness: all three main ratings >= 4
    correct = sum(
        1 for f in feedbacks
        if int(f["tone_rating"]) >= 4 and int(f["helpfulness"]) >= 4 and int(f["personalization"]) >= 4
    )

    accuracy = correct / total
    print(f"✅ Model Tone Accuracy based on user feedback: {accuracy:.2%}")
    return accuracy

compute_feedback_accuracy("user_feedback_log.json")


✅ Model Tone Accuracy based on user feedback: 50.00%
